In [5]:
import requests
import csv
from datetime import datetime
import os

def download_npi_data(updated_since):
    base_url = "https://npiregistry.cms.hhs.gov/api/"
    params = {
        "version": "2.1",
        "updated_since": updated_since.strftime("%Y-%m-%d"),
        "limit": 200,
        "skip": 0
    }

    filename = f"npi_data_since_{updated_since.strftime('%Y%m%d')}.csv"
    
    try:
        with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
            csvwriter = None
            record_count = 0

            while True:
                response = requests.get(base_url, params=params)
                
                if response.status_code != 200:
                    print(f"Error: Received status code {response.status_code}")
                    break
                
                try:
                    data = response.json()
                except ValueError:
                    print("Error: Unable to parse JSON response")
                    break

                results = data.get('results', [])
                
                if not results:
                    break
                
                for result in results:
                    if csvwriter is None:
                        headers = list(result.keys())
                        csvwriter = csv.DictWriter(csvfile, fieldnames=headers)
                        csvwriter.writeheader()

                    flattened_result = {}
                    for key, value in result.items():
                        if isinstance(value, dict):
                            for sub_key, sub_value in value.items():
                                flattened_result[f"{key}_{sub_key}"] = sub_value
                        else:
                            flattened_result[key] = value

                    csvwriter.writerow(flattened_result)
                    record_count += 1
                
                params['skip'] += params['limit']
                print(f"Downloaded {record_count} records so far...")

        if record_count > 0:
            print(f"Data saved to {filename}")
        else:
            print("No data downloaded")
            os.remove(filename)

    except (requests.RequestException, IOError) as e:
        print(f"An error occurred: {e}")

# Set the date to August 5, 2024
date_to_download = datetime.strptime("2024-08-05", "%Y-%m-%d").date()



In [6]:
# Download NPI data since the specified date
download_npi_data(date_to_download)

No data downloaded


In [7]:
import requests
import pandas as pd

def fetch_npi_data():
    # API endpoint
    url = "https://npiregistry.cms.hhs.gov/api/?number=&enumeration_type=&taxonomy_description=&name_purpose=&first_name=&use_first_name_alias=&last_name=&organization_name=&address_purpose=&city=baltimore&state=&postal_code=&country_code=&limit=&skip=&pretty=&version=2.1"
    
    # Make the request to the API
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Extract the results into a DataFrame
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            return df
        else:
            print("No results found in the response.")
            return pd.DataFrame()  # Return an empty DataFrame if no results are found
    else:
        print(f"Error: Received status code {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

# Fetch the data and return it as a pandas DataFrame
npi_df = fetch_npi_data()

# Display the first few rows of the DataFrame
npi_df.head()


,created_epoch,enumeration_type,last_updated_epoch,number,addresses,practiceLocations,basic,taxonomies,identifiers,endpoints,other_names
0,1177362431000,NPI-1,1183948145000,1073731121,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'STEFFI', 'last_name': '.RESNIC...","[{'code': '231H00000X', 'taxonomy_group': '', ...","[{'code': '01', 'desc': 'Other (non-Medicare)'...",[],[]
1,1669930750000,NPI-2,1669930750000,1124738083,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1 ADORING ADULT CARE', ...","[{'code': '310400000X', 'taxonomy_group': '', ...",[],[],[]
2,1691169963000,NPI-2,1695926508000,1174205538,"[{'country_code': 'US', 'country_name': 'Unite...",[],{'organization_name': '1 STOP RECOVERY SERVICE...,"[{'code': '261QM0801X', 'taxonomy_group': '', ...",[],[],[]
3,1459977516000,NPI-2,1459977516000,1578927489,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1046 NORTH POINT, LLC',...","[{'code': '314000000X', 'taxonomy_group': '', ...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...",[],[{'organization_name': 'FUTURECARE NORTHPOINT'...
4,1196436208000,NPI-2,1271796230000,1972785004,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1046 NORTH POINT, LLC',...","[{'code': '314000000X', 'taxonomy_group': '', ...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...",[],[{'organization_name': 'FUTURECARE NORTHPOINT'...


In [9]:
import requests
import pandas as pd

def fetch_npi_data(NPI,city, state=None, postal_code=None, limit=200, skip=0):
    # Base API endpoint
    base_url = "https://npiregistry.cms.hhs.gov/api/"
    
    # Query parameters
    params = {
        "number": "",
        "enumeration_type": "",
        "taxonomy_description": "",
        "name_purpose": "",
        "first_name": "",
        "use_first_name_alias": "",
        "last_name": "",
        "organization_name": "",
        "address_purpose": "",
        "city": city,
        "state": state,
        "postal_code": postal_code,
        "country_code": "",
        "limit": limit,
        "skip": skip,
        "pretty": "",
        "version": "2.1"
    }
    
    # Make the request to the API with the specified parameters
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Extract the results into a DataFrame
        if 'results' in data:
            df = pd.DataFrame(data['results'])
            return df
        else:
            print("No results found in the response.")
            return pd.DataFrame()  # Return an empty DataFrame if no results are found
    else:
        print(f"Error: Received status code {response.status_code}")
        return pd.DataFrame()  # Return an empty DataFrame in case of an error

# Example usage: Fetch NPI data for Baltimore, MD
npi_df = fetch_npi_data(city="Baltimore", state="MD", limit=100)

# Display the first few rows of the DataFrame
npi_df.head(100)


,created_epoch,enumeration_type,last_updated_epoch,number,addresses,practiceLocations,basic,taxonomies,identifiers,endpoints,other_names
0,1177362431000,NPI-1,1183948145000,1073731121,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'STEFFI', 'last_name': '.RESNIC...","[{'code': '231H00000X', 'taxonomy_group': '', ...","[{'code': '01', 'desc': 'Other (non-Medicare)'...",[],[]
1,1669930750000,NPI-2,1669930750000,1124738083,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1 ADORING ADULT CARE', ...","[{'code': '310400000X', 'taxonomy_group': '', ...",[],[],[]
2,1691169963000,NPI-2,1695926508000,1174205538,"[{'country_code': 'US', 'country_name': 'Unite...",[],{'organization_name': '1 STOP RECOVERY SERVICE...,"[{'code': '261QM0801X', 'taxonomy_group': '', ...",[],[],[]
3,1459977516000,NPI-2,1459977516000,1578927489,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1046 NORTH POINT, LLC',...","[{'code': '314000000X', 'taxonomy_group': '', ...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...",[],[{'organization_name': 'FUTURECARE NORTHPOINT'...
4,1196436208000,NPI-2,1271796230000,1972785004,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': '1046 NORTH POINT, LLC',...","[{'code': '314000000X', 'taxonomy_group': '', ...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...",[],[{'organization_name': 'FUTURECARE NORTHPOINT'...
...,...,...,...,...,...,...,...,...,...,...,...
95,1407790576000,NPI-2,1416338711000,1104235456,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'organization_name': 'A. SHAMS PIRZADEH, M.D....","[{'code': '174400000X', 'taxonomy_group': '193...",[],[],[{'organization_name': 'ARTHRITIS AND OSTEPORO...
96,1712839202000,NPI-2,1712839202000,1689427320,"[{'country_code': 'US', 'country_name': 'Unite...",[],{'organization_name': 'A.O. HOLISTIC SUPPORT L...,"[{'code': '101YM0800X', 'taxonomy_group': '193...",[],[],[]
97,1584828140000,NPI-2,1586220459000,1700412723,"[{'country_code': 'US', 'country_name': 'Unite...",[],{'organization_name': 'A.R.P COUNSELING & THER...,"[{'code': '261QM0855X', 'taxonomy_group': '', ...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...","[{'endpointType': 'CONNECT', 'endpointTypeDesc...",[]
98,1610647067000,NPI-2,1692991918000,1114516176,"[{'country_code': 'US', 'country_name': 'Unite...",[],{'organization_name': 'A1 IN-HOME PRIMARY CARE...,"[{'code': '363LF0000X', 'taxonomy_group': '193...","[{'code': '05', 'desc': 'MEDICAID', 'issuer': ...",[],[]
